In [1]:
import numpy as np
import pandas as pd


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss 
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier

pd.set_option('max_columns', None)

In [2]:
train = pd.read_csv('card_dataset/train.csv')
test = pd.read_csv('card_dataset/test.csv')
submission = pd.read_csv('card_dataset/sample_submission.csv')

In [3]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [4]:
train['gen_car_real'] = (train.gender.astype(str)
                         +'_' + train.car.astype(str)+'_'+
                        train.reality.astype(str))

train['phones'] = train.work_phone.astype(str) + '_'+train.phone.astype(str)


test['gen_car_real'] = test.gender.astype(str) +'_'+test.car.astype(str) +'_'+test.reality.astype(str)
test['phones'] = test.work_phone.astype(str)+'_'+test.phone.astype(str)

In [5]:
train.fillna('NAN', inplace = True)
test.fillna('NAN', inplace =True)

In [10]:
new_train = train.drop(['index','gender','car','reality','FLAG_MOBIL','work_phone','phone'],axis = 1)
new_test = test.drop(['index','gender','car','reality','FLAG_MOBIL','work_phone','phone'],axis = 1)

In [11]:
dum_train = pd.get_dummies(new_train)
dum_test = pd.get_dummies(new_test)

In [12]:
dum_train

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,email,family_size,begin_month,credit,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_NAN,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,gen_car_real_F_N_N,gen_car_real_F_N_Y,gen_car_real_F_Y_N,gen_car_real_F_Y_Y,gen_car_real_M_N_N,gen_car_real_M_N_Y,gen_car_real_M_Y_N,gen_car_real_M_Y_Y,phones_0_0,phones_0_1,phones_1_0,phones_1_1
0,0,202500.0,-13899,-4709,0,2.0,-6.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,1,247500.0,-11380,-1540,1,3.0,-5.0,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,0,450000.0,-19087,-4434,0,2.0,-22.0,2.0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,0,202500.0,-15088,-2092,0,2.0,-37.0,0.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,0,157500.0,-15037,-2105,0,2.0,-26.0,2.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,2,225000.0,-12079,-1984,0,4.0,-2.0,1.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
26453,1,180000.0,-15291,-2475,0,2.0,-47.0,2.0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
26454,0,292500.0,-10082,-2015,0,2.0,-25.0,2.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
26455,0,171000.0,-10145,-107,0,1.0,-59.0,2.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [13]:
y = dum_train['credit']
x = dum_train.drop(['credit'], axis = 1)

In [16]:
test_x = dum_test

In [17]:
folds = StratifiedKFold(n_splits = 5, 
                       shuffle = True , random_state = 511)
smote = SMOTE(random_state = 0 )
X_train_over, y_train_over = smote.fit_sample(x, y)

In [20]:
loss = []
sub = np.zeros((test_x.shape[0], 3 ))


for n_folds, (train_index, val_index) in enumerate(folds.split(X_train_over, y_train_over)):
    
    print(f'======================================={n_folds + 1 }==========================================')
    
    X_train, X_val = X_train_over.iloc[train_index], X_train_over.iloc[val_index]
    y_train, y_val = y_train_over.iloc[train_index], y_train_over.iloc[val_index]
    
    
    lgb = LGBMClassifier(n_estimators = 10000,
                        n_jobs = -1, 
                        num_leaves = 2^16,
                        max_depth = 32,
                        random_state = 511,
                        learning_rate = 0.003)
    
    lgb.fit(X_train, y_train,
           eval_set = [(X_train, y_train), (X_val, y_val)],
           early_stopping_rounds = 30,
           verbose  = 1000)
    
    y_val_onehot = pd.get_dummies(y_val)
    prediction = lgb.predict_proba(X_val)
    loss.append(log_loss(y_val_onehot, prediction))
    
    sub+= lgb.predict_proba(test_x)
    
    
    print(f'=============================================================================================\n\n')

=======================================1==========================================
Training until validation scores don't improve for 30 rounds
[1000]	training's multi_logloss: 0.869274	valid_1's multi_logloss: 0.874699
[2000]	training's multi_logloss: 0.798872	valid_1's multi_logloss: 0.811323
[3000]	training's multi_logloss: 0.757013	valid_1's multi_logloss: 0.776348
[4000]	training's multi_logloss: 0.724105	valid_1's multi_logloss: 0.749167
[5000]	training's multi_logloss: 0.696008	valid_1's multi_logloss: 0.726682
[6000]	training's multi_logloss: 0.671272	valid_1's multi_logloss: 0.707905
[7000]	training's multi_logloss: 0.649624	valid_1's multi_logloss: 0.692545
[8000]	training's multi_logloss: 0.629868	valid_1's multi_logloss: 0.677875
[9000]	training's multi_logloss: 0.610837	valid_1's multi_logloss: 0.663705
[10000]	training's multi_logloss: 0.592359	valid_1's multi_logloss: 0.650182
Did not meet early stopping. Best iteration is:
[10000]	training's multi_logloss: 0.592359	vali

In [21]:
np.mean(loss)

0.6563705022043915

In [22]:
submission.iloc[:,1:] = sub/5

In [23]:
submission.to_csv('submission/grouping2.csv',index = False)